# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [ ]:
#include<stdio.h>

int main()
{
    float s = 200, n = 5, j = 1;
    int nS;
    printf("Parcelas do financiamento:");
    for(int i = 0; i<n;i++)
    {   
        if(i>0)
            printf(";");
        nS = s;
        if(s-nS<0.01)
        {
            printf(" %i",nS);
        }
        else
            printf(" %.2f",s);
        s*=(1+(j/100));
    }
    printf("\n");
    
}

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include<stdio.h>

float proximaParcela(float numeroParcela,float valorParcela,float taxa)
{
    if(numeroParcela > 0)
    {
        return calcularParcela(numeroParcela-1,valorParcela*(1+(taxa/100)),taxa);
    }
    return valorParcela;
}


int main()
{
    float s = 200, n = 5, j = 1;
    int nS;
    printf("Parcelas do financiamento:");
    for(int i = 0; i<n;i++)
    {   
        if(i>0)
            printf(";");
        s = proximaParcela(i,200,j);
        nS = s;
        if(s-nS<0.01)
        {
            printf(" %i",nS);
        }
        else
            printf(" %.2f",s);
    }
    printf("\n");
    
}

Parcelas do financiamento: 200; 202; 204.02; 206.06; 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [15]:
/*
Assumindo que esse exercicio se refere a variaveis globais quando diz 
"variáveis não recomendadas que geram dependência dos módulos com o programa principal"
*/

#include<stdio.h>
float s, j;
int n, mes;

float proximaParcela()
{
    mes++;
    s=s*(1+(j/100));
    printf("Parcela %i: %.2f\n",mes,s);
}

float primeiraParcela()
{
    mes = 1;
    printf("Parcela %i: %.2f\n",mes,s);
}


int main()
{
    s = 200, n = 5, j = 1;
    primeiraParcela();
    for(int i = 1; i<n;i++)
    {   
        proximaParcela();
    }
    
}




Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [17]:
#include<stdio.h>

typedef struct {
    float valor;
    int mes;
    float taxa;
} Emprestimo;

float proximaParcela(Emprestimo *emp)
{
        emp->valor = emp->valor*(1+(emp->taxa/100));
        emp->mes -= 1;
}


int main()
{
    Emprestimo emprestimo;
    emprestimo.valor = 200;
    emprestimo.mes = 5;
    emprestimo.taxa = 1;
    
    printf("Parcelas do financiamento:\n");
    printf("valor da parcela 1: %.2f\n",emprestimo.valor);
    emprestimo.mes --;
    int contador = 2;
    while(emprestimo.mes > 0)
    {   
        printf("valor da parcela %d: %.2f\n",contador,proximaParcela(&emprestimo));
        contador++;
    }
}

Parcelas do financiamento:
valor da parcela 1: 200.00
valor da parcela 2: 202.00
valor da parcela 3: 204.02
valor da parcela 4: 206.06
valor da parcela 5: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [ ]:
#include <stdio.h>

typedef struct
{
    float valor;
    int mes;
    float taxa;
}Emprestimo;

void proximaParcela(Emprestimo *emps, int numeroEmprestimos, int mes) // nome original deve ser mostrar parcelas, mas para
{                                                                    // fins de correcao este foi modificado para "proximaParcela"
    for(int i = 0; i < numeroEmprestimos;i++)
    {
        if(emps[i].mes<mes)
            continue;
        printf("Emprestimo %i: parcela %i eh %.2f\n",i+1,mes,emps[i].valor);
        emps[i].valor = emps[i].valor * (1+(emps[i].taxa/100));
    }
}

int main()
{
    int maiorMes;
    int numeroEmprestimos = 2;
    Emprestimo emps[numeroEmprestimos];
    emps[0].valor = 200;
    emps[0].mes = 5;
    emps[0].taxa = 1;
    emps[1].valor = 500;
    emps[1].mes = maiorMes= 7;
    emps[1].taxa = 2;
    for(int i = 0;i<maiorMes;i++)
        mostrarParcelas(emps, numeroEmprestimos, i+1);
    
    
    
}